### validation 결과 확인

In [18]:
import numpy as np
import pandas as pd

targets = pd.DataFrame(columns=['target', 'total', 'train', 'valid', 'f1'])
targets['target'] = np.arange(7)

v3_total = pd.read_csv('../../v3-label_txt/train.csv')
v3_valid_output = pd.read_csv('../../v3-label_txt/valid_output.csv')

v3_ids = list(v3_valid_output['ID'].values)
v3_train = v3_total[~v3_total['ID'].str.contains('|'.join(v3_ids))]
v3_valid_answer = v3_total[v3_total['ID'].str.contains('|'.join(v3_ids))]

label = range(7)
for l in label:
    targets.loc[targets['target']==l, 'total'] = len(v3_total[v3_total['target']==l])
    targets.loc[targets['target']==l, 'train'] = len(v3_train[v3_train['target']==l])
    targets.loc[targets['target']==l, 'valid'] = len(v3_valid_output[v3_valid_output['target']==l])

    val_correct = v3_valid_output[(v3_valid_output['target']==l) & v3_valid_output['ID'].str.contains('|'.join((v3_valid_answer[v3_valid_answer['target']==l]['ID'].values)))]
    recall = len(val_correct) / (len(v3_valid_answer[v3_valid_answer['target']==l]))
    precision = len(val_correct) / (len(v3_valid_output[v3_valid_output['target']==l]))

    targets.loc[targets['target']==l, 'f1'] = 2 * precision * recall / (precision + recall)

targets

,target,total,train,valid,f1
0,0,432,304,136,0.659091
1,1,539,368,135,0.777778
2,2,359,255,110,0.64486
3,3,259,184,37,0.5
4,4,399,288,158,0.60223
5,5,394,268,123,0.626506
6,6,418,293,141,0.691729


### LLaMA~♡

In [ ]:
import json
import pandas as pd

with open('key_maps.json', 'r', encoding='utf-8') as f:
    key_maps = json.load(f)

v3 = pd.read_csv('../../v1.1.2/train.csv')

v3[v3['target']==6].sample(5, random_state=42)

,ID,text,target,polluted_lv
2067,ynat-v1_train_01352,中 위안화 절)행u…고시 R]값l25개월만I 최고,6,0.250000
2082,ynat-v1_train_01376,이스라엘*+레스타인v!v지구 공Y…로L5발사J 대t,6,0.291667
2575,ynat-v1_train_02384,국제 3중XQ4대사관 3안먼 시a 3K주V에 조기 게양,6,0.333333
311,ynat-v1_train_00645,이스라엘 이미 숨진 팔레스타인인 총격 용의자 집까지 파괴,6,0.000000
921,ynat-v1_train_01900,中매체 G20 미중 정상회담 앞두고 평등한 협상이 관건,6,0.100000


In [33]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B' # 'beomi/Llama-3-Open-Ko-8B'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map='auto'
)

model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): L

In [168]:
import pandas as pd
from tqdm.notebook import tqdm

aug = pd.read_csv('../../v1.2.0_aug3/train.csv')
train_fewshot = pd.read_csv('../../v1.1.2/train.csv')

for i in tqdm(range(7), total=7, position=0):
    key = key_maps[str(i)]
    new_aug = pd.DataFrame(columns=['ID', 'text', 'target', 'polluted_lv'])

    PROMPT = f'''당신은 뉴스 기사 제목을 생성해야 합니다. 제공된 키워드, 예시와 유사한 기사 제목을 한 개만 생성하세요.'''

    shots = train_fewshot[(train_fewshot['polluted_lv']<0.4) & (train_fewshot['target']==i)].sample(10, random_state=42)['text'].values
    fewshot = []
    for shot in shots:
        fewshot.append({"role": "user", "content": f"'{key}' 분야에 해당하는 기사 제목을 생성하세요."})
        fewshot.append({"role": "assistant", "content": shot})

    terminators = [
        tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    cnt = 0
    for j in tqdm(range(50), desc='labeling', total=50, position=1, leave=False):
        messages = [{"role": "system", "content": PROMPT}] + fewshot + [{"role": "user", "content": f"'{key}' 분야에 해당하는 기사 제목을 생성하세요."}]
        
        input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors='pt'
        ).to(model.device)

        outputs = model.generate(
            input_ids,
            max_new_tokens=1024,
            eos_token_id=terminators,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            temperature=0.6,
            top_p=0.9
        )

        result = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
        new_aug.loc[j] = [f'augmented-{i}-{j}', result, i, 0]
        # print(result)

    aug = pd.concat([aug, new_aug])
    # print(len(aug))

len(aug)
aug.to_csv('../../v1.2.1/train.csv', index=False)

  0%|          | 0/7 [00:00<?, ?it/s]

labeling:   0%|          | 0/50 [00:00<?, ?it/s]

labeling:   0%|          | 0/50 [00:00<?, ?it/s]

labeling:   0%|          | 0/50 [00:00<?, ?it/s]

labeling:   0%|          | 0/50 [00:00<?, ?it/s]

labeling:   0%|          | 0/50 [00:00<?, ?it/s]

labeling:   0%|          | 0/50 [00:00<?, ?it/s]

labeling:   0%|          | 0/50 [00:00<?, ?it/s]